In [83]:
from time import sleep
from itertools import chain, tee
import base64
from ipyleaflet import *
from IPython.display import display
from ipywidgets import FileUpload
import gpxpy
from PIL import Image
import numpy as np
import cv2

In [56]:
upload = FileUpload(accept='.gpx', multiple=True)
display(upload)

FileUpload(value={}, accept='.gpx', description='Upload', multiple=True)

In [57]:
def get_data(fname):
    points = []
    with open(fname, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append((point.latitude, point.longitude,
                                   point.speed, point.time))
    return points
tracks = [get_data(f) for f in upload.value]

In [58]:
class Extents:
    def __init__(self):
        t = tee(chain.from_iterable(tracks), 4)
        topleft = (min(p[0] for p in t[0]), min(p[1] for p in t[1]))
        botright = (max(p[0] for p in t[2]), max(p[1] for p in t[3]))
        width = botright[0] - topleft[0]
        height = botright[1] - topleft[1]
        self.center = (topleft[0] + 0.5 * width, topleft[1] + 0.5 * height)
        margin = (width * 0.1, height * 0.1)
        self.topleft = (topleft[0] - margin[0], topleft[1] - margin[1])
        self.botright = (botright[0] + margin[0], botright[1] + margin[1])
extents = Extents()

In [72]:
upload = FileUpload(accept='image/*', multiple=False)
display(upload)

FileUpload(value={}, accept='image/*', description='Upload')

In [116]:
class ImgMap:
    def __init__(self, fname):
        self.image_file = Image.open(fname)
        
    def get_data(self):
        data = 'data:image/png;base64,'
        data += base64.b64encode(self.image_file.fp.read()).decode('utf-8')
        return data
    
    def get_size(self):
        return self.image_file.size

    def get_xy(self, lat, lon):
        w, h = self.get_size()
        x = w * (lon - extents.topleft[1]) / (extents.botright[1] - extents.topleft[1])
        y = h * (extents.botright[0] - lat) / (extents.botright[0] - extents.topleft[0])
        return x, y
    
    def get_for_cv(self):
        return cv2.cvtColor(np.array(self.image_file), cv2.COLOR_RGB2BGR)

img_map = ImgMap(next(iter(upload.value)))

In [73]:
class ImageAnchors:
    def __init__(self):
        self.m = Map(center=extents.center, zoom=13)

        image = ImageOverlay(
            url=img_map.get_data(),
            bounds=(extents.topleft, extents.botright),
            opacity=0.6
        )
        self.m.add_layer(image);

        self.image_markers = []
        for lat in (extents.topleft[0], extents.botright[0]):
            for long in (extents.topleft[1], extents.botright[1]):
                mark = Marker(location=(lat, long))
                self.m.add_layer(mark)
                self.image_markers.append(mark)
 
image_anchors = ImageAnchors()
image_anchors.m

Map(center=[50.32000875, 30.57678325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [71]:
class MapAnchors:
    def __init__(self):
        self.m = Map(center=extents.center, zoom=13)

        track = Polyline(
            locations=[[p[0], p[1]] for p in tracks[0]],
            color='red',
            weight=2,
            fill=False,
            opacity=0.6
        )
        self.m.add_layer(track);

        self.image_markers = []
        for lat in (extents.topleft[0], extents.botright[0]):
            for long in (extents.topleft[1], extents.botright[1]):
                mark = Marker(location=(lat, long))
                self.m.add_layer(mark)
                self.image_markers.append(mark)
 
map_anchors = MapAnchors()
map_anchors.m

Map(center=[50.32000875, 30.57678325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [117]:
src_points = np.float32([img_map.get_xy(*p.location) for p in image_anchors.image_markers])
dst_points = np.float32([img_map.get_xy(*p.location) for p in map_anchors.image_markers])
mat = cv2.getPerspectiveTransform(src_points, dst_points)
img = img_map.get_for_cv()
img = cv2.warpPerspective(img, mat, img_map.get_size())
is_ok, png_map = cv2.imencode('.png', img)
is_ok

True

In [118]:
class FinalMap:
    def __init__(self):
        self.m = Map(center=extents.center, zoom=13)

        data = 'data:image/png;base64,'
        data += base64.b64encode(png_map).decode('utf-8')

        image = ImageOverlay(
            url=data,
            bounds=(extents.topleft, extents.botright),
            opacity=0.6
        )
        self.m.add_layer(image);

        track = Polyline(
            locations=[[p[0], p[1]] for p in tracks[0]],
            color='red',
            weight=2,
            fill=False,
            opacity=0.6
        )
        self.m.add_layer(track);

final_map = FinalMap()
final_map.m

Map(center=[50.32000875, 30.57678325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [10]:
nb_steps = 10
step = 0.1
start_location = mark.location
end_location = (52.2, 10)

for i in range(nb_steps + 1):
    percentage_animation = i / nb_steps
    
    a = 1 - percentage_animation
    b = percentage_animation
    
    mark.location = (a * start_location[0] + b * end_location[0], a * start_location[1] + b * end_location[1])
    
    sleep(step)
